In [9]:
from util import *
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,\
    Embedding, Lambda
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
 
# Define the corpus
# corpus = ['The cat sat on the mat',
#           'The dog ran in the park',
#           'The bird sang in the tree']
train_corpus = open("dataset/train_clean.txt", "r",encoding='utf8').read().split(".")
train_noDiactric, train_labels =extract_data(train_corpus)
# Convert the corpus to a sequence of integers
tokenizer = Tokenizer(oov_token=1)
tokenizer.fit_on_texts(train_noDiactric)
train_sequences = tokenizer.texts_to_sequences(train_noDiactric)
word2idx = tokenizer.word_index
idx2word = {v:k for k, v in word2idx.items()}
word2idx['UKN']= len(word2idx) +1

In [10]:
freq=tokenizer.word_counts
least_10_words = sorted(freq.items(), key=lambda x: x[1])[:10]
# print(least_10_words)
for word, _ in least_10_words:
    # print(word, freq)
    del word2idx[word]

In [11]:

val_corpus = open("dataset/val_clean.txt", "r",encoding='utf8').read().split(".")
val_noDiactric, val_labels =extract_data(val_corpus)
# Convert the corpus to a sequence of integers
val_sequences = tokenizer.texts_to_sequences(val_noDiactric)
# val_sequences= []
# for i,sentence in enumerate(val_noDiactric):
#     val_sequences.append(list())
#     for word in sentence:
#         if word not in word2idx:
#             val_sequences[i].append(word2idx['UKN'])
#         else: 
#             val_sequences[i].append(word2idx[word])

In [12]:
print(word2idx['قوله'])
print(val_noDiactric[0])
print(train_noDiactric[0])
print(val_sequences[0])
print(train_sequences[0])

9
 قوله : ولا تكره ضيافته
قوله : ( أو قطع الأول يده إلخ ) قال الزركشي ابن عرفة : قوله : بلفظ يقتضيه كإنكار غير حديث بالإسلام وجوب ما علم وجوبه من الدين ضرورة ( كإلقاء مصحف بقذر وشد زنار ) ابن عرفة : قول ابن شاس : أو بفعل يتضمنه هو كلبس الزنار وإلقاء المصحف في صريح النجاسة والسجود للصنم ونحو ذلك ( وسحر ) محمد : قول مالك وأصحابه أن الساحر كافر بالله تعالى قال مالك : هو كالزنديق إذا عمل السحر بنفسه قتل ولم يستتب
[9, 23, 4333, 26212]
[9, 5, 279, 93, 194, 49, 15, 911, 39, 466, 9, 1194, 3657, 41322, 41, 255, 4169, 483, 13, 168, 1752, 4, 163, 1186, 27897, 6115, 41323, 5844, 41324, 39, 466, 65, 39, 2665, 5, 1373, 24303, 46, 17908, 57698, 41325, 2975, 3, 1444, 2163, 9596, 57699, 1199, 20, 21706, 89, 65, 97, 3374, 7, 16446, 1200, 1195, 113, 15, 97, 46, 27898, 30, 529, 8306, 421, 270, 53, 57700]


In [13]:
# Define the parameters
# I think change to +2?
# vocab_size = len(tokenizer.word_index) + 1
vocab_size = len(tokenizer.word_index) + 2
embedding_size = 20 # was originally 10
window_size = 3 
 
# Generate the context-target pairs
contexts = []
targets = []
for sequence in train_sequences:
    for i in range(window_size, len(sequence) - window_size):
        context = sequence[i - window_size:i] +\
            sequence[i + 1:i + window_size + 1]
        target = sequence[i]
        contexts.append(context)
        targets.append(target)
val_conexts = []
val_targets = []
for sequence in val_sequences:
    for i in range(window_size, len(sequence) - window_size):
        context = sequence[i - window_size:i] +\
            sequence[i + 1:i + window_size + 1]
        target = sequence[i]
        val_conexts.append(context)
        val_targets.append(target)
# Convert the contexts and targets to numpy arrays
X_train = np.array(contexts)
Y_train = np.array(targets)
X_val = np.array(val_conexts)
Y_val = np.array(val_targets)

 
# Define the CBOW model
model = Sequential()
model.add(Embedding(input_dim=vocab_size,
                    output_dim=embedding_size,
                    input_length=2*window_size))
model.add(Lambda(lambda x: tf.reduce_mean(x, axis=1)))
model.add(Dense(units=vocab_size, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam')
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 6, 20)             2152140   
                                                                 
 lambda (Lambda)             (None, 20)                0         
                                                                 
 dense (Dense)               (None, 107607)            2259747   
                                                                 
Total params: 4411887 (16.83 MB)
Trainable params: 4411887 (16.83 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
history = model.fit(X_train, Y_train, epochs=30, batch_size=20, validation_data=(X_val, Y_val))

MemoryError: Unable to allocate 39.6 GiB for an array with shape (98862, 107604) and data type float32

In [ ]:
model.save_weights('cbow_weights.h5')
 
 
# Load the pre-trained weights
model.load_weights('cbow_weights.h5')

In [ ]:

# # Get the word embeddings
# embeddings = model.get_weights()[0]
 
# # Perform PCA to reduce the dimensionality
# # of the embeddings
# pca = PCA(n_components=2)
# reduced_embeddings = pca.fit_transform(embeddings)
 
# # Visualize the embeddings
# plt.figure(figsize=(5, 5))
# for i, word in enumerate(tokenizer.word_index.keys()):
#     x, y = reduced_embeddings[i]
#     plt.scatter(x, y)
#     plt.annotate(word, xy=(x, y), xytext=(5, 2),
#                  textcoords='offset points',
#                  ha='right', va='bottom')
# plt.show()